In [ ]:
from nbdev import *
%nbdev_default_export examples

Cells will be exported to pct.examples,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_hide
%reload_ext autoreload
%autoreload 2

# Examples

> Examples of the PCT library in use.

In [ ]:
# hide
import sys
sys.path.append("..")

In [ ]:
import gym 
import os

In [ ]:
os = os.name

## Cartpole

0       Cart Position             
1       Cart Velocity             
2       Pole Angle                
3       Pole Angular Velocity 
        


In [ ]:
from pct.hierarchy import PCTHierarchy
from pct.utilities import FunctionsList
from pct.functions import OpenAIGym
from pct.functions import IndexedParameter
from pct.functions import Integration
from pct.functions import GreaterThan
from pct.functions import PassOn

Create a hierarchy of 4 levels each with one node.

In [ ]:
cartpole_hierarchy = PCTHierarchy(levels=4, cols=1, name="cartpoleh", build=False)

In [ ]:
cartpole_hierarchy.summary(build=False)

cartpoleh PCTHierarchy
**************************
PRE: None
Level 0
level0col0 PCTNode
----------------------------
REF: constant Constant | 1 | links  proportional1 
PER: variable Variable | 0 
COM: subtract Subtract | 0 | links  constant variable 
OUT: proportional Proportional | gain 10 | 0 | links  subtract 
----------------------------
Level 1
level1col0 PCTNode
----------------------------
REF: constant1 Constant | 1 | links  proportional2 
PER: variable1 Variable | 0 | links  variable 
COM: subtract1 Subtract | 0 | links  constant1 variable1 
OUT: proportional1 Proportional | gain 10 | 0 | links  subtract1 
----------------------------
Level 2
level2col0 PCTNode
----------------------------
REF: constant2 Constant | 1 | links  proportional3 
PER: variable2 Variable | 0 | links  variable1 
COM: subtract2 Subtract | 0 | links  constant2 variable2 
OUT: proportional2 Proportional | gain 10 | 0 | links  subtract2 
----------------------------
Level 3
level3col0 PCTNode
-------------

In [ ]:
cartpole = OpenAIGym("CartPole-v1", name="CartPole-v1", render=True)

Create functions for each of the observation parameters of the Cartpole environment. Insert them into the hierarchy at the desired places.

In [ ]:
cartpole_hierarchy.insert_function(level=0, col=0, collection="perception", function=IndexedParameter(index=1, name="cart_velocity", links=[cartpole]))
cartpole_hierarchy.insert_function(level=1, col=0, collection="perception", function=IndexedParameter(index=0, name="cart_position", links=[cartpole]))
cartpole_hierarchy.insert_function(level=2, col=0, collection="perception", function=IndexedParameter(index=3, name="pole_velocity", links=[cartpole]))
cartpole_hierarchy.insert_function(level=3, col=0, collection="perception", function=IndexedParameter(index=2, name="pole_angle", links=[cartpole]))

Link the references to the outputs of the level up.

In [ ]:
cartpole_hierarchy.insert_function(level=0, col=0, collection="reference", function=PassOn(name="cart_velocity_reference", links=['proportional1']))
cartpole_hierarchy.insert_function(level=1, col=0, collection="reference", function=PassOn(name="cart_position_reference", links=['proportional2']))
cartpole_hierarchy.insert_function(level=2, col=0, collection="reference", function=PassOn(name="pole_velocity_reference", links=['proportional3']))

In [ ]:
cartpole_hierarchy.summary(build=False)

Set the highest level reference.

In [ ]:
top = cartpole_hierarchy.get_function(level=3, col=0, collection="reference")
top.set_name("pole_angle_reference")
top.set_value(0)

Link the output of the hierarchy back to the Cartpole environment.

In [ ]:
cartpole_hierarchy.insert_function(level=0, col=0, collection="output", function=Integration(gain=2, slow=10, name="force", links='subtract'))

Set the names and gains of the output functions. This also shows another way of getting a function, by name.

In [ ]:
FunctionsList.getInstance().get_function("proportional3").set_name("pole_angle_output")
FunctionsList.getInstance().get_function("pole_angle_output").set_property('gain', 3)

FunctionsList.getInstance().get_function("proportional2").set_name("pole_velocity_output")
FunctionsList.getInstance().get_function("pole_velocity_output").set_property('gain', 2)

FunctionsList.getInstance().get_function("proportional1").set_name("cart_position_output")
FunctionsList.getInstance().get_function("cart_position_output").set_property('gain', 1)

FunctionsList.getInstance().get_function("force").set_property('gain', 1)

Add the cartpole function as one that is executed before the actual hierarchy.

In [ ]:
cartpole_hierarchy.add_preprocessor(cartpole)

Add a post function to convert the output to 1 or 0 as required by the Cartpole environment. 

In [ ]:
cartpole_hierarchy.add_postprocessor(GreaterThan(threshold=0, upper=0, lower=1, links='force'))

In [ ]:
link = cartpole_hierarchy.get_output_function()
cartpole.add_link(link)

In [ ]:
cartpole_hierarchy.summary(build=False)

Sit back and observe the brilliance of your efforts. 

In [ ]:
cartpole_hierarchy.summary()

In [ ]:
cartpole_hierarchy.set_order("Down")
#cartpole_hierarchy.run(100,verbose=False)

In [ ]:
cartpole_hierarchy(verbose=True)

In [ ]:
cartpole.close()